<header style="padding:1px;background:#f9f9f9;border-top:3px solid #00b2b1"><img id="Teradata-logo" src="https://www.teradata.com/Teradata/Images/Rebrand/Teradata_logo-two_color.png" alt="Teradata" width="220" align="right" />

<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>Predictive Maintenance using Vantage</b>
</header>

<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>Introduction:</b></p>
<p style = 'font-size:16px;font-family:Arial'>Predictive maintenance of production lines is important to early detect possible defects and thus identify and apply the required maintenance activities to avoid possible breakdowns. An important concern in predictive maintenance is the prediction of remaining useful life (RUL), which is an estimate of the number of remaining years that a component in a production line is estimated to be able to function in accordance with its intended purpose before warranting replacement.</p>

<center><img src="images/turbofan.jpg" alt="TurboFan" width=400 height=400/></center>
<p>image source: <a href="https://unsplash.com/photos/OjxpywWo9HI">unsplash.com</a></p>

<p style='font-size:16px;font-family:Arial'>To achieve the goal of predict the RUL (Remaining Useful Life) of turbofan engines, We will be leveraging the power of <b>Teradata Vantage</b>, an advanced analytics platform. With Teradata Vantage, we can deploy machine learning algorithms through teradataml python library, which enable us to identify and mitigate potential machine failures before they even occur.</p>
<p style='font-size:16px;font-family:Arial'>Teradata Vantage provides us with the necessary capabilities to analyze the vast amounts of data collected for turbofan engines , such as engine number, sensor measurement, and operational settings. By processing this data and detecting anomalies or patterns, we can take proactive measures to address potential issues, preventing costly downtimes and ensuring the longevity of the machines.</p>
<p style='font-size:16px;font-family:Arial'>With Teradata Vantage, we can help to client stay ahead of the curve, providing them with cutting-edge analytics capabilities to improve the reliability and efficiency of their machines.</p>

<p style = 'font-size:16px;font-family:Arial;color:#E37C4D'><b>Steps in the analysis:</b></p>
<ol style = 'font-size:16px;font-family:Arial'>
    <li>Configuring the environment</li>
    <li>Connect to Vantage</li>
    <li>Data Exploration</li>
    <li>Data Preparation</li>
    <li>Train-Test Split</li>
    <li>In-Database Machine Learning</li>
    <li>In-Database Model Scoring</li>
    <li>Visualize the results</li>
    <li>Cleanup</li>
</ol>

<hr>
<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>1. Configuring the environment</b>
<p style = 'font-size:16px;font-family:Arial'>Here, we import the required libraries, set environment variables and environment paths (if required).</p>

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
import plotly.graph_objs as go

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, roc_auc_score, roc_curve, auc
from sklearn.preprocessing import label_binarize
from teradataml import *
import warnings

configure.val_install_location = 'val'
display.max_rows = 10

# Filter out warnings
warnings.filterwarnings('ignore')

<hr>
<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>2. Initiate a connection to Vantage</b>
<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'> <b>Let's start by connecting to the Teradata system </b></p>
<p style = 'font-size:16px;font-family:Arial'>You will be prompted to provide the password. Enter your password, press the Enter key, and then use the down arrow to go to the next cell.</p>

In [ ]:
%run -i ../startup.ipynb
eng = create_context(host = 'host.docker.internal', username='demo_user', password = password)
print(eng)
eng.execute('''SET query_band='DEMO=UseCases/Predictive_Maintenance/Remaining_Useful_Life_Forecasting_PY_SQL.ipynb;' UPDATE FOR SESSION;''')

<p style = 'font-size:16px;font-family:Arial'>Begin running steps with Shift + Enter keys. </p>

<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>Getting Data for This Demo</b></p>
<p style = 'font-size:16px;font-family:Arial'>We have provided data for this demo on cloud storage. You can either run the demo using foreign tables to access the data without any storage on your environment or download the data to local storage, which may yield faster execution. Still, there could be considerations of available storage. Two statements are in the following cell, and one is commented out. You may switch which mode you choose by changing the comment string.</p>

In [ ]:
%run -i ../run_procedure.py "call get_data('DEMO_RemaingUsefulLife_cloud');"        # Takes 1 minute
# %run -i ../run_procedure.py "call get_data('DEMO_RemaingUsefulLife_local');"        # Takes 2 minutes

<p style = 'font-size:16px;font-family:Arial'>Next is an optional step – if you want to see the status of databases/tables created and space used.</p>

In [ ]:
%run -i ../run_procedure.py "call space_report();"        # Takes 10 seconds

<hr>
<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>3. Data Exploration</b>

<p style = 'font-size:16px;font-family:Arial'>The goal of the Predictive Maintenance System (PAM) is to predict the RUL (Remaining Useful Life) of turbofan engines.</p>
<p style = 'font-size:16px;font-family:Arial'>The PAM system will predict the number of remaining operational cycles before failure in the test set. i.e., the number of operational cycles after the last cycle that the engine will continue to operate. 
Also provided a vector of true Remaining Useful Life (RUL) values for the test data.</p>

<p style = 'font-size:16px;font-family:Arial'>The data is from 100 engines, with hundreds of rows of measurements collected from each time the engine was cycled. There are three operational settings that have a substantial effect on engine performance. Data is collected from 27 sensors. There is a row for every time the engine is cycled (starting at 1) and a column counting down the cycles until maintenance is required, labelled RUL, or Remaining Useful Life.<p/>


<p style = 'font-size:16px;font-family:Arial'>Each row is a snapshot of data taken during a single operational cycle, each column is a different variable. 
The columns correspond to:</p>
<ol style = 'font-size:16px;font-family:Arial'>
    <li>unit number</li>
    <li>time, in cycles</li>
    <li>operational setting 1</li>
    <li>operational setting 2</li>
    <li>operational setting 3</li>
    <li>sensor measurement 1</li>
    <li>sensor measurement 2</li>
    <li>...</li>
    <li>sensor measurement 27</li>


</ol>

<p style = 'font-size:16px;font-family:Arial'>The data from <a href=https://www.kaggle.com/c/predictive-maintenance>https://www.kaggle.com/c/predictive-maintenance</a> is loaded in Vantage in a table named "predictive_maintenance_rul".</p>

<p style = 'font-size:16px;font-family:Arial'><b><i>*Please scroll down to the end of the notebook for detailed column descriptions of the dataset.</i></b></p>

<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>3.1 Examine the predictive maintenance RUL table</b></p>    
<p style = 'font-size:16px;font-family:Arial'>Let's look at the sample data in the predictive_maintenance_rul table.</p>

In [ ]:
df = DataFrame(in_schema('DEMO_RemaingUsefulLife', 'predictive_maintenance_rul'))
df.sort('id')

<p style = 'font-size:16px;font-family:Arial'>Here is a list of columns with their data types and non-null record counts.</p>

In [ ]:
df.info(null_counts=True)

<p style = 'font-size:16px;font-family:Arial'>We have total 34 variables, out of these 28 variables are of type Float and rest 6 are Integer </p>

<p style = 'font-size:16px;font-family:Arial'>Now, let's do some data exploration with engine number and time_in_cycle variables.</p>

In [ ]:
query = '''
SELECT
    engine_no,
    COUNT(time_in_cycles) AS #cycles_completed
FROM
    DEMO_RemaingUsefulLife.predictive_maintenance_rul
GROUP BY 1
ORDER BY 2 DESC;
'''

df_eng_cycles = pd.read_sql(query, eng)
df_eng_cycles.head(10)

In [ ]:
def get_histogram(df, x, y, title, x_title, y_title, width=1200, height=500):
    fig = px.histogram(df, x=x, y =y, title=title, nbins=df.shape[0])
    fig.update_yaxes(title=y_title)
    fig.update_xaxes(title=x_title)
    fig.update_layout(
        autosize=False,
        width=width,
        height=height,)
    return fig

In [ ]:
get_histogram(df_eng_cycles, 
              x="engine_no", 
              y = "#cycles_completed", 
              title="Number of cycles completed until maintenance needed by Engine Number", 
              x_title="engine_no", 
              y_title="Number of cycles completed")

<p style = 'font-size:16px;font-family:Arial'>We see that engine numbers <b>69, 92, 96, and 67</b> have more than 300 cycles completed, which is higher than other engines.</p>
<p style = 'font-size:16px;font-family:Arial'>Let's see the average number of cycles completed per engine.</p>

In [ ]:
query = '''
SELECT AVG(avg_cycles_completed) FROM (
    SELECT
    engine_no,
    COUNT(1) AS avg_cycles_completed
    FROM
        DEMO_RemaingUsefulLife.predictive_maintenance_rul
    GROUP BY 1
) AS t;
'''

pd.read_sql(query, eng)

<p style = 'font-size:16px;font-family:Arial'>We can see that the top engine number <b>69</b> has completed approximately 54% more cycles than the average of all the engines.</p>

<hr>
<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>4. Data Preparation</b>

<p style = 'font-size:16px;font-family:Arial'><b>We'll perform the following steps:</b></p>
<ul style = 'font-size:16px;font-family:Arial'>
    <li>Missing Value Analysis</li>
    <li>Data distribution plot for numerical variables.</li>
    <li>Features selection using correlation</li>   
    <li>Features scaling using z-score</li>

</ul>


<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>4.1 Missing Value analysis</b></p>

In [ ]:
from teradataml import ColumnSummary
def get_missing_values_table(df):
    obj = ColumnSummary(data = df, target_columns=df.columns)
    return obj.result[['ColumnName','NullCount','NullPercentage']].sort('NullPercentage', ascending=False)

In [ ]:
get_missing_values_table(df)

<p style = 'font-size:16px;font-family:Arial'>From the above results, we see null values in the columns sensors_22 to sensors_27. Although these columns have 100% null values,</p>

In [ ]:
# Drop columns with null
df = df.drop(columns=['sensor_22','sensor_23','sensor_24','sensor_25','sensor_26','sensor_27'])

<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>4.2 Distribution plots for numeric variables</b></p>

In [ ]:
def get_distribution_plot(df, cols):
    plotnumber = 1
    h,l,c = 5, len(cols), 6 
    r = int(np.ceil(l/c))
    plt.figure(figsize = (20, 3*r))
    for col in cols:
        if plotnumber <= l:
            ax = plt.subplot(r, c, plotnumber)
            plt.hist(df[[col]].get_values())
            plt.xlabel(col, fontsize = 12)
        plotnumber += 1
    plt.tight_layout()
    plt.show()

In [ ]:
get_distribution_plot(df, df.columns)

<p style = 'font-size:16px;font-family:Arial'>From the above results, we see some of the features like <b>sensor_1, sensor_5, sensor_10, .etc</b> values are unique. They will not contribute any values to our model development.</p>

<p style = 'font-size:16px;font-family:Arial'>So, Let's drop that columns here.</p>

In [ ]:
# Drop columns all the unique values
df = df.drop(columns=['sensor_1','sensor_5','sensor_10','sensor_16','sensor_18','sensor_19','op_setting_3'])

<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>4.3 Checking the correlation</b></p>

<p style = 'font-size:16px;font-family:Arial'>Here, we'll check the correlation of all the numeric features. Measuring correlation lets you
    determine if the value of one variable is useful in predicting the value of another.</p>
    
<p style = 'font-size:16px;font-family:Arial'>
The Sample Pearson product moment correlation coefficient is a measure of the linear association between variables. The boundary on the computed coefficient ranges from -1.00 to +1.00.</p>

<p style = 'font-size:16px;font-family:Arial'>
Note that high correlation does not imply a causal relationship between the variables. The following table indicates the meaning of four extreme values for the coefficient of correlation between two variables.
</p>

<table style = 'font-size:16px;font-family:Arial'>
    <th>IF the correlation coefficient has this value</th>
    <th>THEN the association between the variables</th>
    <tr>
        <td>-1.00</td>
        <td>is perfectly linear, but inverse. <br>
        As the value for y varies, the value for x varies identically in the opposite direction.</td>
    </tr>
    <tr>
        <td>0</td>
        <td>does not exist and they are said to be uncorrelated.</td>
    </tr>
     <tr>
        <td>+1.00</td>
        <td>is perfectly linear.<br>
        As the value for y varies, the value for x varies identically in the same direction..</td>
    </tr>
</table>

In [ ]:
def get_heatmap(df):
    # heatmap
    corr = df.to_pandas().corr()
    mask = np.triu(np.ones_like(corr, dtype = bool))
    fig = px.imshow(corr, text_auto='.2f', width=1100, height=1100, aspect="auto", color_continuous_scale=["lightblue","lightyellow"])
    return fig.show()

In [ ]:
get_heatmap(df)

<p style = 'font-size:16px;font-family:Arial'>Few observations from the correlation matrix above are:</p>
<ul style = 'font-size:16px;font-family:Arial'>
    <li>Time in cycle and remaining useful life (RUL) have a negative correlation with a value of -0.74.</li>
    <li>Sensor 2 and Sensor 12 have a -0.72 correlation, while Sensor 12 and Sensor 7 have a 0.81 correlation.</li>
    <li>Sensor 12 has a negative correlation with Sensor 11 and Sensor 4, with values of -0.85 and -0.82, respectively.</li>
    <li>Sensor 4 has a positive correlation with Sensor 11 and Sensor 13, with values of 0.83 and 0.75, respectively.</li>
</ul>

<p style = 'font-size:16px;font-family:Arial'>While these correlations exist, we cannot train good models from the dataset. So as remedies, we can drop one column out of two columns with a higher correlation.</p>

In [ ]:
# Drop columns all the unique values
df_sel_ft = df.drop(columns=['time_in_cycles','sensor_2','sensor_4','sensor_7','sensor_8','sensor_9','sensor_11','sensor_12','sensor_15','sensor_20'])

<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>4.4 Pairplot for multivariate correlations</b></p>

In [ ]:
# Create a pairplot to visualize multivariate correlations
sns.pairplot(df_sel_ft.to_pandas()[["op_setting_1", "op_setting_2","sensor_3","sensor_14","sensor_13","RUL"]], diag_kind = 'auto', hue = 'RUL', palette="crest")

<p style = 'font-size:16px;font-family:Arial'> From the above pairplot, we can observe the variations in each plot. The plots are in matrix format where the row name represents x axis and column name represents the y axis. The main-diagonal subplots are the univariate histograms (distributions) for each attribute.</p>

<p style = 'font-size:16px;font-family:Arial'>In the above pairplot, we are doing analysis of few sensors and operational settings with remaining useful life values. RUL values are divided in 5 ranges like 0-79, 80-159, etc.</p>
<p style = 'font-size:16px;font-family:Arial'>While comparing 2 values in pair along with RUL ranges colors from light green to dark blue, we observe a few things:</p>
<ol style = 'font-size:16px;font-family:Arial'>
    <li>When observing the 5th row with 1st column values, We can see a higher side value of RUL (dark blue) when op_setting_1 is less than 0.00 and sensor_13 values are less than 1.0.</li>
    <li>If we check the 5th-row 3rd column - when sensor_3 values are between 1570 and 1600, and sensor_13 values below 0.3 the RUL values are between 240 and 320 units.</li>
    <li>When we observe the 4th row with the 4th column we can see that data distributions in column sensor_14 are right skewed.</li>
    <li>We can see that data distributions is normal in the 1st row 1st column op_setting_1.</li>
</ol>

<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>4.5 ZScore </b></p>

<p style = 'font-size:16px;font-family:Arial'>Z-Score transforms each column value into the number of standard deviations from the mean value of the column.</p>

In [ ]:
# create ZScore obj
zs = ZScore(columns = df_sel_ft.columns[1:-1],
            out_columns = df_sel_ft.columns[1:-1])

# list of columns to retain in output
retain = Retain(columns = 'RUL')

# Process the transformation
df_transformed = valib.Transform(
                            data = df_sel_ft, 
                            zscore = zs, 
                            index_columns = "id",
                            key_columns = "id",
                            retain=retain
                         )

In [ ]:
df_transformed.result.to_sql(
                "rul_transformed_data",
                schema_name = "demo_user",
                primary_index="id",
                if_exists="replace"
            )

<p style = 'font-size:16px;font-family:Arial'>Save the transformed dataframe into a table <b>rul_transformed_data</b>.</p>

<hr>
<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>5. Train-Test Split</b>
<p style = 'font-size:16px;font-family:Arial'>In the next step, we'll split the transformed dataset into training and testing datasets in the ratio 80:20, and we will save the datasets into Vantage.</p>

<p style = 'font-size:16px;font-family:Arial'>Post spliting the dataset into train/test. Let's see number of records in train and test.</p>

In [ ]:
query = f'''CREATE MULTISET TABLE TrainTestSplit_output AS (
    SELECT * FROM TD_TrainTestSplit(
        ON rul_transformed_data AS InputTable
        USING
        IDColumn('id')
        trainSize(0.80)
        testSize(0.20)
        Seed(7)
    ) AS dt
) WITH DATA;'''

try:
    eng.execute(query)
except:
    eng.execute('DROP TABLE TrainTestSplit_output;')
    eng.execute(query)

In [ ]:
query = f'''CREATE MULTISET TABLE rul_train AS (
    SELECT * FROM TrainTestSplit_output WHERE TD_IsTrainRow = 1
) WITH DATA;'''

try:
    eng.execute(query)
except:
    eng.execute('DROP TABLE rul_train;')
    eng.execute(query)

In [ ]:
query = f'''CREATE MULTISET TABLE rul_test AS (
    SELECT * FROM TrainTestSplit_output WHERE TD_IsTrainRow = 0
) WITH DATA;'''

try:
    eng.execute(query)
except:
    eng.execute('DROP TABLE rul_test;')
    eng.execute(query)

In [ ]:
df_train = DataFrame('rul_train')
df_test = DataFrame('rul_test')
print("Training Set = "+str(df_train.shape[0])+". Testing Set = "+str(df_test.shape[0]))

<hr>
<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>6. In-Database Machine Learning</b>

<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>6.1 Train a XGBoost Model</b></p>

<p style = 'font-size:16px;font-family:Arial'>In the next step, we'll use the TD_XGBOOST function to train an xgboost model using the RUL column as the target variable for regression. XGBoost's tree-based ensemble approach, regularization techniques, handling of missing values, scalability, and feature importance capabilities make it a powerful and effective choice for modeling tabular data, often leading to superior performance compared to other machine learning algorithms.
<br>
<br>
The TD_XGBoost function, eXtreme Gradient Boosting, implements the gradient-boosted decision tree designed for speed and performance. It has recently been dominating applied machine learning.
<br>
<br>
In gradient boosting, each iteration fits a model to the residuals (errors) of the previous iteration to correct the errors made by existing models. The predicted residual is multiplied by this learning rate and then added to the previous prediction. Models are added sequentially until no further improvements can be made. It is called gradient boosting because it uses a gradient descent algorithm to minimize the loss when adding new models.
</p>

In [ ]:
# Create a table xgb_model using TD_XGBoost from Teradata
# The TD_XGBoost function partitions the data by any column, trains an XGBoost regression model with default trees, 
# maximum depth of 5, and 10 iterations, and saves the output to a metadata table xgb_out.
# If the table xgb_model already exists, drop it and the metadata table xgb_out before creating the new table.

query = f'''CREATE TABLE xgb_model AS (
SELECT * FROM TD_XGBoost(
ON rul_train PARTITION BY ANY
OUT TABLE MetaInformationTable(xgb_out) 
USING
    ResponseColumn('RUL')
    InputColumns('engine_no', 'op_setting_1', 'op_setting_2', 'sensor_3', 'sensor_6','sensor_13', 'sensor_14', 'sensor_17', 'sensor_21')
    MaxDepth(5)
    NumBoostedTrees(-1)
    ModelType('regression')
    Seed(123)
    ShrinkageFactor(0.1)
    IterNum(10) 
    ColumnSampling(1.0) 
) AS dt) WITH DATA;
'''

try:
    eng.execute(query)
except Exception as e:
    # Drop the tables and try again if the table already exists
    eng.execute(f'DROP TABLE xgb_model;')
    eng.execute(f'DROP TABLE xgb_out;')
    eng.execute(query)

<hr>
<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>6.2 XGBoost - Model Scoring</b></p>
<p style = 'font-size:16px;font-family:Arial'>
In the next step, we'll use the TD_XGBoostPredict function to score the xgboost model trained in the previous step.</p>

In [ ]:
query = '''CREATE TABLE xgb_predict_out AS (
SELECT * FROM TD_XGBoostPredict(
ON rul_test AS inputtable PARTITION BY ANY
ON xgb_model AS modeltable DIMENSION ORDER BY task_index, tree_num, iter, tree_order
USING
    IdColumn('id')
    ModelType('regression')
    Accumulate('RUL')
) AS dt) WITH DATA;
'''

try:
    eng.execute(query)
except Exception as e:
    eng.execute('DROP TABLE xgb_predict_out;')
    eng.execute(query)

In [ ]:
xgb_result = DataFrame(in_schema('demo_user', 'xgb_predict_out'))

In [ ]:
xgb_result_pd=xgb_result.to_pandas().reset_index().sort_values("id").rename(columns={'RUL':'Actual'})
xgb_result_pd

<p style = 'font-size:16px;font-family:Arial'>Next, we'll use the TD_RegressionEvaluator function to evaluate the trained xgboost model on test data. This will let us know how well our model has performed on unseen data.</p>

In [ ]:
# Evaluate the XGBoost model's performance using TD_RegressionEvaluator
# Check if the necessary tables exist before executing the query

if not eng.has_table('xgb_predict_out'):
    print('Error: xgb_predict_out table does not exist.')
    sys.exit(1)

query = '''SELECT * FROM TD_RegressionEvaluator(
ON xgb_predict_out as InputTable
USING
ObservationColumn('RUL')
PredictionColumn('prediction')
Metrics('MAE','MSE','RMSE','R2','FSTAT')
DegreesOfFreedom(1,28)
NUMOFINDEPENDENTVARIABLES(15)
) as dt;
'''

try:
    XGB_Eval = pd.read_sql(query, eng)
except:
    eng.execute('DROP TABLE additional_metrics;')
    eng.execute(query)


In [ ]:
XGB_Eval['model']='XGBoost'
XGB_Eval

<p style = 'font-size:16px;font-family:Arial'>The result table displays the evaluation metrics for XGBoost models retrieved from TD_RegressionEvaluator. The lower the RMS error value, the better the model's performance. Here F_conclusion is <b>Fail to reject null hypothesis</b> which means that our XGBoost model is not fitting perfectly on our data. In other words, none of the predictor variables have a statistically significant relationship with the response variable, RUL.</p>

<hr/>

<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>6.3 Train a Decision Forest Model</b></p>

<p style = 'font-size:16px;font-family:Arial'>The Decision Forest is a powerful method used for predicting outcomes in both classification and regression problems. It's an improvement on the technique of combining (or "bagging") multiple decision trees. Normally, building a decision tree involves assessing the importance of each feature in the data to determine how to divide the information. This method takes a unique approach by only considering a random subset of features at each division point in the tree. This forces each decision tree within the "forest" to be different from one another, which ultimately improves the accuracy of the predictions. The function relies on a training dataset to develop a prediction model. Then, the TD_DecisionForestPredict function uses the model built by the TD_DecisionForest function to make predictions. It supports regression, binary, and multi-class classification tasks.</p>

<p style = 'font-size:16px;font-family:Arial'>Typically, constructing a decision tree involves evaluating the value for each input feature in the data to select a split point. The function reduces the features to a random subset (that can be considered at each split point); the algorithm can force each decision tree in the forest to be very different to improve prediction accuracy. The function uses a training dataset to create a predictive model. The TD_DecisionForestPredict function uses the model created by the TD_DecisionForest function for making predictions. The function supports regression, binary, and multi-class classification.</p>

<p style = 'font-size:16px;font-family:Arial'>Consider the following points:
<li style = 'font-size:16px;font-family:Arial'>All input features are numeric. Convert the categorical columns to numerical columns as preprocessing step.</li>
<li style = 'font-size:16px;font-family:Arial'>For classification, class labels (ResponseColumn values) can only be integers. A maximum of 500 classes is supported for classification.</li>
<li style = 'font-size:16px;font-family:Arial'>Observations with missing values in any input column will be ignored during training. To fill in missing values, use the TD_SimpleImpute function.</li>
<li style = 'font-size:16px;font-family:Arial'>The number of trees built by the TD_DecisionForest function depends on the values of NumTrees, TreeSize, and CoverageFactor, as well as the data distribution in the cluster. The trees are built simultaneously by all the processing units (AMPs) that have a non-empty portion of the data.</li>
</p>


In [ ]:
query = '''Create table DF_train as (
SELECT * FROM TD_DecisionForest (
    ON rul_train AS INPUTTABLE partition by ANY
USING
    ResponseColumn('RUL')
    InputColumns('engine_no', 'op_setting_1', 'op_setting_2', 'sensor_3', 'sensor_6','sensor_13', 'sensor_14', 'sensor_17', 'sensor_21')
    MaxDepth(12)
    MinNodeSize(1)
    NumTrees(4)
    ModelType('REGRESSION')
    Seed(1)
    Mtry(-1)
    MtrySeed(1)
) AS dt
) with data;
'''
try:
    eng.execute(query)
except:
    eng.execute('DROP TABLE DF_train;')
    eng.execute(query)

<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>6.4 Decision Forest - Model Scoring</b></p>
<p style = 'font-size:16px;font-family:Arial'>
In the next step, we'll use the TD_DecisionForestPredict function to score the decision forest model trained in the previous step.</p>

In [ ]:
query = '''
Create table DF_Predict as (
SELECT * FROM TD_DecisionForestPredict (
ON rul_test AS InputTable PARTITION BY ANY
ON DF_Train AS ModelTable DIMENSION
USING
  IdColumn ('id')
  Detailed('false')
  Accumulate('RUL')
) AS dt) with data;'''

try:
    eng.execute(query)
except:
    eng.execute('DROP TABLE DF_Predict;')
    eng.execute(query)

In [ ]:
df_result = DataFrame(in_schema('demo_user', 'DF_Predict'))
df_result_pd=df_result.to_pandas().reset_index().sort_values("id").rename(columns={'RUL':'Actual'})
df_result_pd

<p style = 'font-size:16px;font-family:Arial'>The TD_RegressionEvaluator function computes metrics to evaluate and compare multiple models and summarizes how close predictions are to their expected values.</p>

In [ ]:
query = '''
SELECT * FROM TD_RegressionEvaluator(
ON DF_Predict as InputTable
USING
ObservationColumn('RUL')
PredictionColumn('prediction')
Metrics('MAE','MSE','RMSE','R2','FSTAT')
DegreesOfFreedom(5,28)
NUMOFINDEPENDENTVARIABLES(15)
) as dt;
'''

DF_eval=pd.read_sql(query, eng)


In [ ]:
DF_eval['model']='DecisionForest'
DF_eval

<p style = 'font-size:16px;font-family:Arial'>The result table displays the evaluation metrics for DecisionForest models retrieved from TD_RegressionEvaluator. The lower the RMS error value, the better the model's performance. Here F_conclusion is <b>Reject null hypothesis</b> which means that our DecisionForest model is fitting perfectly on our data. In other words, the predictor variables like sensors, op_sesstoins, etc. have a statistically significant relationship with the response variable, RUL</p>

<hr>
<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>8. Visualize the results</b>

<p style = 'font-size:16px;font-family:Arial'>The Metrics of the regression evaluator has the RMSE, R2 and the F-STAT metrics which are specified in the Metrics.</p>

<p style = 'font-size:16px;font-family:Arial'>Thus here we have used 2 different models to train and predict the data. The Regression evaluator is used to evaluate and compare the models. The Teradata In-Database functions are used for training, prediction and evaluation. In this case since we have sample data the result parameters may not be accurate for these models.</p>  

<p style = 'font-size:16px;font-family:Arial'>Root mean squared error (RMSE)The most common metric for evaluating linear regression model performance is called root mean squared error, or RMSE. The basic idea is to measure how bad/erroneous the model’s predictions are when compared to actual observed values. So a high RMSE is “bad” and a low RMSE is “good”.</p>

<p style = 'font-size:16px;font-family:Arial'>The coefficient of determination — more commonly known as R² — allows us to measure the strength of the relationship between the response and predictor variables in the model. It’s just the square of the correlation coefficient R, so its values are in the range 0.0–1.0. Higher values of R- Squared is Good.</p>

<p style = 'font-size:16px;font-family:Arial'>The metrics specified in the Metrics syntax element are displayed. For FSTAT, the following columns are displayed:</p>
<li style = 'font-size:16px;font-family:Arial'>F_score</li>
<li style = 'font-size:16px;font-family:Arial'>F_Critcialvalue</li>
<li style = 'font-size:16px;font-family:Arial'>p_value</li>
<li style = 'font-size:16px;font-family:Arial'>F_Conclusion.</li></p>

<p style = 'font-size:16px;font-family:Arial'>Here we can see the comparison for MAE,MSE,RMSE and R2 for XGBoost and DecisionForest.</p> 


In [ ]:
frames=[DF_eval,XGB_Eval]
result = pd.concat(frames)
result = result.set_index([['Decision Forest','XGBoost']])
result = result.drop(['model'],axis=1)
transposed_df_eval = result.transpose()
transposed_df_eval

<p style = 'font-size:16px;font-family:Arial'>The above metrics compare the Decision Forest and XGBoost models. We can see that Decision Forest is performing better than XGBoost. Also, F_conclusion is Reject null hypothesis which means that our DecisionForest model is fitting perfectly on our data.</p>

<p style = 'font-size:16px;font-family:Arial'>Let's visualise the the Decision Forest prediction result to compare actual vs. predicted values in graph.</p>

In [ ]:
# taking fist 50 records
df_result_sample = df_result_pd[:50]
plt.figure(figsize=(20,8))
plt.xlabel('Remaining Useful Life', fontsize = 14)
plt.ylabel('ID', fontsize = 14)
plt.plot(df_result_sample['id'], df_result_sample['Actual'], color='g', label='Actual Value')
plt.plot(df_result_sample['id'], df_result_sample['prediction'], color='r', label='Predicted Value')
plt.title('Actual vs Predicted using DecisionForest Classification', fontsize = 18)
plt.legend()
# plt.ylim(0,1)
plt.show()

<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>Conclusion:</b></p>
<p style = 'font-size:16px;font-family:Arial'>In conclusion, the implementation of a predictive maintenance solution can greatly benefit to the client by reducing machine downtime and maintenance costs, improving production efficiency, and increasing overall productivity. Proactive scheduling of maintenance based on real-time data and analytics can help prevent costly breakdowns and emergency repairs, leading to improved machine reliability.
    <br>
    <br>
Additionally, setting limits and alarms on key parameters can enable early detection of potential failures, allowing for timely maintenance interventions. The ability to predict the type of failure can also help reduce diagnosis time, further optimizing maintenance efforts. By leveraging predictive maintenance, client can make data-driven decisions to improve their maintenance strategy, leading to tangible benefits to the company's bottom line, including increased operational efficiency, reduced costs, and improved overall performance.</p>

<hr>
<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>9. Cleanup</b>
<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>Work Tables</b></p>
<p style = 'font-size:16px;font-family:Arial'>Cleanup work tables to prevent errors next time.</p>

In [ ]:
db_drop_table(table_name="rul_transformed_data")

In [ ]:
db_drop_table(table_name="rul_train")

In [ ]:
db_drop_table(table_name="rul_test")

In [ ]:
db_drop_table(table_name="TrainTestSplit_output")

In [ ]:
db_drop_table(table_name="xgb_out")

In [ ]:
db_drop_table(table_name="xgb_model")

In [ ]:
db_drop_table(table_name="xgb_predict_out")

<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'> <b>Databases and Tables </b></p>
<p style = 'font-size:16px;font-family:Arial'>The following code will clean up tables and databases created above.</p>

In [ ]:
%run -i ../run_procedure.py "call remove_data('DEMO_Retail');"        # Takes 5 seconds

In [ ]:
remove_context()

<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>Dataset:</b>

- `engine_no`: Unique identifier ranging from 1 to 100
- `index`: Unique row id
- `time_in_cycles`: the number of operational cycles
- `op_setting_1`: operational settings1 that have a substantial effect on engine performance.
- `op_setting_2`: operational settings2 that have a substantial effect on engine performance.
- `op_setting_3`: operational settings3 that have a substantial effect on engine performance.
- `sensor_1`: sensor measurement 1
- `sensor_2`: sensor measurement 2
- `sensor_3`: sensor measurement 3
- `sensor_4`: sensor measurement 4
- `sensor_5`: sensor measurement 5
- `sensor_6`: sensor measurement 6
- `sensor_7`: sensor measurement 7
- `sensor_8`: sensor measurement 8
- `sensor_9`: sensor measurement 9
- `sensor_10`: sensor measurement 10
- `sensor_11`: sensor measurement 11
- `sensor_12`: sensor measurement 12
- `sensor_13`: sensor measurement 13
- `sensor_14`: sensor measurement 14
- `sensor_15`: sensor measurement 15
- `sensor_16`: sensor measurement 16
- `sensor_17`: sensor measurement 17
- `sensor_18`: sensor measurement 18
- `sensor_19`: sensor measurement 19
- `sensor_20`: sensor measurement 20
- `sensor_21`: sensor measurement 21
- `sensor_22`: sensor measurement 22
- `sensor_23`: sensor measurement 23
- `sensor_24`: sensor measurement 24
- `sensor_25`: sensor measurement 25
- `sensor_26`: sensor measurement 26
- `sensor_27`: sensor measurement 27
- `RUL`: predict the number of remaining operational cycles before failure in the test set, 
i.e., the number of operational cycles after the last cycle that the engine will continue to operate. 
Also provided a vector of true Remaining Useful Life (RUL) values for the test data.


<footer style="padding:10px;background:#f9f9f9;border-bottom:3px solid #394851">Copyright © Teradata Corporation - 2023. All Rights Reserved.</footer>